In [1]:
%load_ext autoreload

%autoreload 3

In [2]:
%reload_ext autoreload

In [9]:
!pip install wget
!python -m wget https://chromedriver.storage.googleapis.com/108.0.5359.71/chromedriver_win32.zip

You should consider upgrading via the 'C:\Users\user\.virtualenvs\google-arts-0o2XoGZ3\Scripts\python.exe -m pip install --upgrade pip' command.



Saved under chromedriver_win32.zip


# Packages

In [49]:
import pyprojroot
from pathlib  import Path
from src.scraping.scraping_pictures import ScrapingPictures
import time

# Paths

In [65]:
root_path =  pyprojroot.here()
path_driver = (root_path / "notebooks"/'chromedriver.exe').relative_to('/')
data_raw_folder = (root_path / "data"/'raw')

In [63]:

scraper = ScrapingPictures(path_driver)
scraper.open("WHITE")
scraper.scroll_down(0)
time.sleep(2)
df = scraper.scrape()
#print(src)

Open page section WHITE
Finish scrolling down


# Export Picture Data

In [67]:
df.to_csv(data_raw_folder/'pictures.csv',index=False)